In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [2]:
from springstone.data import get_data, get_missing_dates, create_train_test, create_df_for_prophet
from springstone.utils import bollinger_bands, moving_average
import pandas as pd

In [3]:
hist = get_data('TSLA')

[*********************100%***********************]  1 of 1 completed


In [4]:
hist = bollinger_bands(hist, 'Close', 20, 2)
hist = moving_average(hist, 'Close', 7)

In [5]:
hist

,Open,High,Low,Close,Volume,Close_bb20_2,Close_ma7
Date,,,,,,,
2012-02-21,6.974000,6.974000,6.762000,6.900000,5679000,NaN,NaN
2012-02-22,6.900000,6.944000,6.500000,6.844000,8273000,NaN,NaN
2012-02-23,6.798000,6.994000,6.712000,6.906000,4102000,NaN,NaN
2012-02-24,6.846000,6.904000,6.654000,6.750000,4799500,NaN,NaN
2012-02-27,6.682000,6.800000,6.600000,6.724000,3030000,NaN,NaN
...,...,...,...,...,...,...,...
2022-02-14,861.570007,898.880005,853.150024,875.760010,22585500,1019.233524,903.567147
2022-02-15,900.000000,923.000000,893.380005,922.429993,19216500,999.264296,903.440002
2022-02-16,914.049988,926.429993,901.210022,923.390015,17098100,986.864937,905.732858


In [6]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import numpy as np

In [7]:
df_prophet = create_df_for_prophet(hist)
df_train, df_test = create_train_test(df_prophet)

In [8]:
df_train

,ds,y
0,2012-02-21,6.900000
1,2012-02-22,6.844000
2,2012-02-23,6.906000
3,2012-02-24,6.750000
4,2012-02-27,6.724000
...,...,...
2009,2020-02-14,160.005997
2010,2020-02-18,171.679993
2011,2020-02-19,183.483994
2012,2020-02-20,179.882004


In [9]:
m_prophet = Prophet()
m_prophet.fit(df_train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -109.46
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       4538.81     0.0655047       1089.05           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        4562.5     0.0547564       308.701           1           1      225   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        4568.1    0.00107922       74.2597           1           1      340   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       4569.19   3.59004e-05       69.6651      0.4271      0.4271      473   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       4570.96   2.94842e-05        64.515       0.448       0.448      593   
    Iter      log prob        ||dx||      ||grad||       alpha   

   4571.3   6.44679e-06       73.5244      0.3562      0.3562      724   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     699       4571.35   0.000431844       82.6309           1           1      851   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     799       4571.47   0.000475628       66.8941       0.686       0.686      970   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     899       4571.56   1.16894e-06        53.768      0.4896      0.4896     1105   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     908       4571.56   1.88895e-06       59.1886   2.855e-08       0.001     1154  LS failed, Hessian reset 
     912       4571.56   9.49471e-08       57.3271      0.2513           1     1161   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tol

In [10]:
future = m_prophet.make_future_dataframe(periods=30)
future

,ds
0,2012-02-21
1,2012-02-22
2,2012-02-23
3,2012-02-24
4,2012-02-27
...,...
2039,2020-03-18
2040,2020-03-19
2041,2020-03-20
2042,2020-03-21


In [11]:
forecast = m_prophet.predict(future)
forecast = forecast[-30:][['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [12]:
forecast

,ds,yhat,yhat_lower,yhat_upper
2014,2020-02-22,79.841538,66.310490,93.648362
2015,2020-02-23,79.279480,65.373291,92.986427
2016,2020-02-24,77.153813,63.814299,91.409462
2017,2020-02-25,76.782357,62.358948,90.657941
2018,2020-02-26,76.073343,62.471523,89.819947
2019,2020-02-27,75.655782,61.833525,88.710032
2020,2020-02-28,74.920374,61.528426,88.990437
2021,2020-02-29,75.817127,62.069266,89.884243
2022,2020-03-01,75.291586,61.468860,88.643127
2023,2020-03-02,73.250568,58.883773,87.346132


In [13]:
df_test[:30]

,ds,y
2014,2020-02-24,166.757996
2015,2020-02-25,159.981995
2016,2020-02-26,155.759995
2017,2020-02-27,135.800003
2018,2020-02-28,133.598007
2019,2020-03-02,148.723999
2020,2020-03-03,149.102005
2021,2020-03-04,149.899994
2022,2020-03-05,144.908005
2023,2020-03-06,140.695999


In [14]:
mae_prophet = mean_absolute_error(df_test[:30]['y'], forecast['yhat'])
mae_prophet

43.14107735193851

In [29]:
rows = get_missing_dates(hist, True)
an_array = np.full((rows.shape[0], hist.shape[1]), np.nan)
an_array.shape

(1133, 7)

In [32]:
non_business_days = pd.DataFrame(an_array, index=rows, columns=hist.columns)

In [ ]:
pd.concat(hist, non_business_days)